# Beaconing activity detection model

The objective of the this notebook is to analyze events to search for potentional beaconing using a statistical approach.  

First, we shall import the required libraries:

In [ ]:
import pandas as pd
import numpy as np
from dateutil.parser import parse
from elasticsearch import Elasticsearch

Now we import the .csv file that contains sysmon events collected over a period of approximately two days.

In [ ]:
username = 'elastic'
password = 'changeme'

# Connect to Elasticsearch
es = Elasticsearch([{'host': '34.67.212.1', 'port': 9200, 'scheme': 'http'}])

query = {
  "size": 10000,
  "query": {
    "range": {
      "@timestamp": {
        "gte": "now-24h/h",
        "lt": "now/h"
      }
    }
  }
}


# Execute the query
result = es.search(index='.ds-winlogbeat-*', body=query)

# Extract data from the result
hits = result['hits']['hits']
df = pd.DataFrame([hit['_source'] for hit in hits])

print(df.shape[0])
df.head()

We're currently interested in three columns: the IP addresses of the destination and source, and the time at which each interaction between each unique IP pair took place.

We will also save the most recent SystemTime for later use.

In [ ]:
# Expand the dictionaries into separate columns
df_destination = df['destination'].apply(pd.Series)
df_source = df['source'].apply(pd.Series)
df_process = df['process'].apply(pd.Series)

# Rename the columns to avoid name conflicts
df_destination = df_destination.rename(columns=lambda x: 'destination.' + str(x))
df_source = df_source.rename(columns=lambda x: 'source.' + str(x))
df_process = df_process.rename(columns=lambda x: 'process.' + str(x))

# Concatenate the expanded columns back to the original DataFrame
df = pd.concat([df.drop(['destination', 'source'], axis=1), df_destination, df_source, df_process], axis=1)

# Now, filter the DataFrame to include only the target columns
target_columns = ["destination.ip", "source.ip", "@timestamp", "process.name", "process.executable"]
df = df[target_columns]

# Drop rows with missing (NaN) values
df = df.dropna()

print(df.shape[0])

df_original = df.copy()
df.head(10)

In [ ]:
# Group the data by SourceIp and DestinationIp, and calculate the total number of connections and the list of connection times for each unique IP pair
df = (
    df.groupby(["source.ip", "destination.ip"])
    .agg(
        TotalConnections=pd.NamedAgg(column="@timestamp", aggfunc="count"),
        ConnectionTimes=pd.NamedAgg(column="@timestamp", aggfunc=list),
    )
    .reset_index()
)

print("Number of unique host pairs:", df.shape[0])

df.head(10)

Now that we have created a dataframe with unique IP pairs as well as the total number of connections for each pair and the datetime of each connection, we move to calculate three scores:
* Skew score
* MAD score
* Count score

Let's start with the Skew(ness) score

### 1. Skewness score

For each IP pair, we need to: 
* Calculate time intervals between all consecutive connections 
* Calculate the skewness of these time intervals: A skewness score close to zero could indicate regular, periodic activity, while a high skewness score might suggest more varied or irregular intervals.

In [ ]:
def compute_bowleys_skewness(connection_times):
    # Convert connection times to numpy array
    connection_times = np.array(connection_times)

    # Sort the connection times and compute intervals (differences)
    connection_times.sort()
    diffs = np.diff(connection_times).astype("timedelta64[s]").astype(int)

    # Data cleansing: Filter out intervals less than 1 second and check if enough data remains
    diffs = diffs[diffs > 1]
    if len(diffs) < 6:
        return None  # Not enough data to compute a score

    # Calculate the quartiles
    Q1, Q2, Q3 = np.percentile(diffs, [25, 50, 75])

    # Compute Bowley's skewness
    bowleys_numerator = Q1 + Q3 - 2 * Q2
    bowleys_denominator = Q3 - Q1
    if bowleys_denominator == 0 or Q2 == Q1 or Q2 == Q3:
        return 0  # Handle division by zero and identical quartile cases

    bowleys_skewness = bowleys_numerator / bowleys_denominator

    # Skew score is the complement of the absolute value of Bowley's skewness to 1
    skew_score = 1.0 - abs(bowleys_skewness)
    return skew_score

# Convert and sort timestamp strings to datetime objects in the "ConnectionTimes" column.
df["ConnectionTimes"] = df["ConnectionTimes"].apply(
    lambda x: sorted([parse(t) for t in x])
)

# Now apply the `compute_bowleys_skewness` function to each list of connection times
df["Skew score"] = df["ConnectionTimes"].apply(compute_bowleys_skewness)

df.head()

### 2. MAD Score

In [ ]:
def compute_mad_score(connection_times):
    # Convert connection times to numpy array
    connection_times = np.array(connection_times)

    # Sort the connection times and compute intervals (differences)
    connection_times.sort()
    diffs = np.diff(connection_times).astype("timedelta64[s]").astype(int)

    # Data cleansing: Filter out intervals less than 1 second and check if enough data remains
    diffs = diffs[diffs > 1]
    if len(diffs) < 6:
        return None  # Not enough data to compute a score

    # Calculate the median of the differences
    tsMid = np.percentile(diffs, 50)

    # Calculate the Median Absolute Deviation (MAD) about the median
    tsMadm = np.median(np.abs(diffs - tsMid))

    # Normalize the MAD score
    tsMadmScore = 1.0 - float(tsMadm) / 30.0

    # Ensure the MAD score is non-negative
    tsMadmScore = max(tsMadmScore, 0)

    return tsMadmScore

# Apply the compute_mad_score function to each list of connection times
df["MAD score"] = df["ConnectionTimes"].apply(compute_mad_score)

df.head()

### 3. Count score

In [ ]:
def compute_connection_count_score(connection_times):
    if len(connection_times) < 6:
        return None  # Not enough data to compute a meaningful score

    connection_times.sort()
    # Calculate the total time span of observed connections in seconds
    time_span_seconds = (connection_times[-1] - connection_times[0]).total_seconds()
    # Calculate the median interval in seconds between connections
    diffs = np.diff(connection_times).astype("timedelta64[s]").astype(int)
    tsMid = np.percentile(diffs, 50)

    # Avoid division by zero in case tsMid is 0
    if tsMid == 0:
        return 0

    # Calculate the expected number of connections based on the median interval and the actual time span
    tsTimespanDiv = float(time_span_seconds) / tsMid
    # Calculate the connection count score based on the actual and expected number of connections
    tsConnCountScore = float(len(connection_times)) / tsTimespanDiv

    # Cap the score at 1.0
    tsConnCountScore = min(tsConnCountScore, 1.0)

    return tsConnCountScore

df["Count score"] = df["ConnectionTimes"].apply(compute_connection_count_score)

df.head()

### 4. Compute the final score

In [ ]:
def compute_combined_score(row):
    return round(((row["Skew score"] + row["MAD score"] + row["Count score"]) / 3), 4)

df = df.drop(columns=["ConnectionTimes"])

# Drop rows with missing values
df.dropna(inplace=True)

# Compute the combined score for each unique host pair
df["Score"] = df.apply(compute_combined_score, axis=1)

# Sort the dataframe by the score in descending order
df.sort_values("Score", ascending=False, inplace=True)

print('number of rows', df.shape[0])

df.head()

In [ ]:
df_with_details = df.merge(
    df_original[
        ["destination.ip", "source.ip", "@timestamp", "process.name", "process.executable"]
    ],
    on=["source.ip", "destination.ip"],
    how="left",
)

print(df_with_details.columns)

df.drop_duplicates(
    subset=["source.ip", "destination.ip"], keep="first", inplace=True
)

df = (
    df_with_details.groupby(["source.ip", "destination.ip"])
    .agg(
        {
            "TotalConnections": "first",
            "Score": "first",
            "Skew score": "first",
            "MAD score": "first",
            "Count score": "first",
            "process.name": lambda x: " | ".join(
                set(x.dropna().astype(str))
            ),  # Join unique fileNames after converting to string
            "process.executable": lambda x: " | ".join(
                set(x.dropna().astype(str))
            ),  # Join unique commandLines after converting to string
        }
    )
    .reset_index()
)

df.sort_values("Score", ascending=False, inplace=True)

df.to_csv("./Results/final_results.csv")

print(df.shape[0])

df.head(10)